### student scrapper activity
scrape categories and all books for each category and then insert into mongo db

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pymongo

In [2]:
browser = Browser('chrome')

In [3]:
def get_soup(url):
    browser.visit(url)
    html = browser.html    
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [4]:
def get_categories():
    soup = get_soup('http://books.toscrape.com/')
    
    sidebar = soup.find('ul', class_='nav nav-list')
    catogeries_html = sidebar.ul.find_all('a')
    categories = []
    for category in catogeries_html:
        data = { 'category' : category.text.strip() ,
        'href' : 'http://books.toscrape.com/' + category['href']
        }
        categories.append(data)
    return categories;

In [5]:
def get_articles():
    html = browser.html    
    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.find_all('article', class_='product_pod')
    return articles

In [6]:
def get_books_in_article(url , articles):
    books = []
    for article in articles:
        book ={
            'title' : article.h3.a.text,
            'href' : urljoin(url,article.h3.a['href'])
        }
        books.append(book)
    return books

In [7]:
def scrape_all_books(url):
    more_books = True
    first_iterations = True
    iterations = 0
    books = []
    while more_books:
#         soup = get_soup(url)
        iterations += 1
        if first_iterations:
            browser.visit(url)
            first_iterations = False
        else:
            pass
        articles = get_articles()
        books_for_article = get_books_in_article(url, articles)
        books.append(books_for_article)
        #         if iterations > 3:
#             more_books=False
        more_books=True
        try:
            browser.links.find_by_partial_text('next').click()          
        except:
#             print("Scraping Complete")
            more_books = False
    return books

In [8]:
categories = get_categories()

In [9]:
for category in categories:
    print(f"Scrapping for {category['category']}....")
    books = scrape_all_books(category['href'])
    category['books'] = books

Scrapping for Travel....
Scrapping for Mystery....
Scrapping for Historical Fiction....
Scrapping for Sequential Art....
Scrapping for Classics....
Scrapping for Philosophy....
Scrapping for Romance....
Scrapping for Womens Fiction....
Scrapping for Fiction....
Scrapping for Childrens....
Scrapping for Religion....
Scrapping for Nonfiction....
Scrapping for Music....
Scrapping for Default....
Scrapping for Science Fiction....
Scrapping for Sports and Games....
Scrapping for Add a comment....
Scrapping for Fantasy....
Scrapping for New Adult....
Scrapping for Young Adult....
Scrapping for Science....
Scrapping for Poetry....
Scrapping for Paranormal....
Scrapping for Art....
Scrapping for Psychology....
Scrapping for Autobiography....
Scrapping for Parenting....
Scrapping for Adult Fiction....
Scrapping for Humor....
Scrapping for Horror....
Scrapping for History....
Scrapping for Food and Drink....
Scrapping for Christian Fiction....
Scrapping for Business....
Scrapping for Biography..

In [10]:
browser.quit()

In [11]:
def save_books(books):
    conn = 'mongodb://localhost:27017'
    client = pymongo.MongoClient(conn)
    db = client.books
    collection = db.books
    collection.insert_many(categories)


In [12]:
save_books(categories)